---
## The homophily and social contagion of cheating

In this project, we will study the homophily and social contagion of cheating in the massive multiplayer online game PlayerUnknown's Battlegrounds (PUBG). Cheating in this context means the adoption of unapproved software that gives the player an unfair advantage in the game (e.g. being able to see through walls). 

Our hypotheses are that cheaters tend to associate with other cheaters but also, players who interact with cheaters become likely to adopt cheating themselves. To provide preliminary evidence for these hypotheses, we will:

1. Observe whether cheaters tend to team up with other cheaters more than chance.
2. Observe whether players who observe cheaters are likely to become cheaters more than chance.
3. Observe whether players who are killed by cheaters are likely to become cheaters more than chance.

To test the "more than chance" part, we will simulate alternative universes in which the players played the same game but joined a different team or happened to be killed by someone else at a different time. We will then compare how what we observe in the actual data compares to what we would expect in a "randomized" world.  

For this project, I developed all data processing using core Python only — no external libraries like pandas or numpy.




### Data

The data contains the following files:

* `cheaters.txt` – contains cheaters who played between March 1 and March 10, 2019
    1. player account id
    2. estimated date when the player started cheating
    3. date when the player's account was banned due to cheating


* `kills.txt` – contains the killings done in 6,000 randomly selected matches played between March 1 and March 10, 2019
    1. match id 
    2. account id of the killer
    3. account id of the player who got killed
    4. time when the kill happened
 
 
* `team_ids.txt` – contains the team ids for players in 5,419 team-play matches in the same period. If a match from the kills.txt file does not appear in these data, we will assume that it was in single-player mode.  
    1. match id 
    2. player account id
    3. team id in match


## Importing the code

In [1]:
# Import modules here
from data_processing import *
from team_analysis import *
from player_interactions import *
from statistical_methods import *


### 1. Do cheaters team up?

Use the files `cheaters.txt` and `team_ids.txt` to estimate how often cheaters (regardless of when exactly they started cheating) end up on the same team. Your output should say how many teams have 0, 1, 2, 3, or 4 cheaters.

Now, randomly shuffle the team ids among the players in a match. Repeat this 20 times and estimate the expected counts as before. Output the mean and the 95% confidence intervals for the expected counts. 

*Optional: Conclude in a short comment what you observe. This reflection is optional and will not be marked.*

In [2]:

# Do cheaters team up more often than expected?

# Loading in the data
cheaters = load_cheaters('../data/cheaters.txt')
team_dict, match_team_player = load_team_data('../data/team_ids.txt')

# Determining how often cheaters end up on the same team
actual_counts(cheaters, team_dict)

#Randomly shuffling the team assignments and counting the number of times cheaters end up on the same team
cheater_count_random = randomised_cheater_counts(match_team_player, cheaters, number_of_shuffles=20)

#Computing and printing mean and confidence intervals for the expected counts
_= calculate_mean_and_confidence_intervals(cheater_count_random, output_format="detailed")




Actual counts:
Teams with 0 cheaters: 170782
Teams with 1 cheaters: 3199
Teams with 2 cheaters: 182
Teams with 3 cheaters: 9
Teams with 4 cheaters: 2
Randomised counts:
Teams with 0 cheaters: Mean = 170608.5, 95% CI = (170605.9, 170611.1)
Teams with 1 cheaters: Mean = 3533.2, 95% CI = (3528.0, 3538.4)
Teams with 2 cheaters: Mean = 32.1, 95% CI = (29.59, 34.61)


### 2. Do victims of cheating start cheating?

Use the files `cheaters.txt` and `kills.txt` to count how many players got killed by an active cheater on at least one occasion and then started cheating. Specifically, we are interested in situations where:

1. Player B has started cheating but player A is not cheating.
2. Player B kills player A.
3. At some point afterwards, player A starts cheating.

Output the count in the data. 

Then, simulate alternative worlds in which everything is the same but the events took somewhat different sequence. To do so, randomize within a game, keeping the timing and structure of interactions but shuffling the player ids. Generate 20 randomizations like this and estimate the expected count of victims of cheating who start cheating as before. Output the mean and the 95% confidence interval for the expected count in these randomized worlds.

*Optional: Conclude in a short comment what you observe. This reflection is optional and will not be marked.*

#### Hint

Starting time of cheating is estimated as a date, so assume that a player cheats on any match that they started playing on that date or afterwards. Use the first kill in a match to determine the match starting date. Thus, if the first kill in the match occurred before midnight of the cheating date but the match ended after midnight, we will assume that the player was not cheating just yet. 


In [3]:

#Loading in the data
cheaters = parse_cheaters('../data/cheaters.txt')  # Load cheaters data
kills = parse_kills('../data/kills.txt')  # Load kills data

# Setting the match start times to the first kill of each match
match_start_times = get_match_start_times_from_kills(kills)

# Counting and printing the number of situations where all the conditions are true
count = count_true_conditions(kills, cheaters, match_start_times)
print(f"Number of situations where both conditions are true: {count}")

# Simulating alternate worlds with shuffled player ids and storing the randomisations
victims_start_cheating_counts, stored_randomisations = simulate_victim_cheating_events(20, kills, cheaters)

# Computing and printing the mean and confidence intervals of expected counts
_=calculate_mean_and_confidence_intervals(victims_start_cheating_counts)




Number of situations where both conditions are true: 47
Randomized counts for starting cheating:
Expected Count (Mean) = 49.25, 95% CI = (46.42, 52.08)


### 3. Do observers of cheating start cheating?

Use the files `cheaters.txt` and `kills.txt` to count how many players observed an active cheater on at least one occasion and then started cheating. Cheating players can be recognized because they exhibit abnormal killing patterns. We will assume that player A realizes that player B cheats if:

1. Player B has started cheating but player A is not cheating.
2. Player B kills at least 3 other players before player A gets killed in the game.
3. At some point afterwards, player A starts cheating.

Output the count in the data.

Then, use the 20 randomizations from Part 2 to estimate the expected count of observers of cheating who start cheating. Output the mean and the 95% confidence interval for the expected count in these randomized worlds.

*Optional: Conclude in a short comment what you observe. This reflection is optional and will not be marked.*

In [4]:

# Counting instances where observers of cheating start cheating
real_observer_count = get_observers_become_cheaters(kills, cheaters, match_start_times)
print(f"Number of real-world observers who started cheating: {real_observer_count}")

#Using stored randomisations from Question 2 to estimate expected count of observers who start cheating
observer_cheating_counts = simulate_observer_cheating_events(stored_randomisations, cheaters, match_start_times)

#Computing and printing the mean and confidence intervals of expected counts
_=calculate_mean_and_confidence_intervals(observer_cheating_counts)

    

Number of real-world observers who started cheating: 215
Randomized counts for starting cheating:
Expected Count (Mean) = 230.85, 95% CI = (225.6, 236.1)
